In [1]:
from PIL import Image
from src.circle_detection import detect_circles, hough_circle_transform, draw_detected_circles, np2img
from src.edge_detection import edge_detection, compress_image, apply_gaussian_blur, compute_gradients, non_maximum_suppression, double_threshold
import os

def list_image_names(directory):
    """
    列出指定目录中所有图片文件的名称，不包括子目录。
    """
    images = []
    supported_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
    for file in os.listdir(directory):
        if file.endswith(supported_extensions):
            images.append(file)
    return images


directory_path = './data/test_images'
results_path = './data/results'
images = list_image_names(directory_path)

for image_name in images:
    if not os.path.exists(f'{results_path}/{image_name}'):
        os.makedirs(f'{results_path}/{image_name}')
    
    image = Image.open(f'./data/test_images/{image_name}').convert('RGB')
    image = compress_image(image, base_width=400)  # 调整图像到固定大小，提高后续参数的可用性
    blurred_image = apply_gaussian_blur(image)  # 参数可调
    blurred_image.save(f'{results_path}/{image_name}/blurred_image.png')

    gradient_magnitude, gradient_direction = compute_gradients(blurred_image)
    np2img(gradient_magnitude).save(f'{results_path}/{image_name}/gradient.png')

    nms_image = non_maximum_suppression(gradient_magnitude, gradient_direction)
    edges = double_threshold(nms_image, low_threshold=70 , high_threshold=90)  # 参数可调
    np2img(edges).save(f'{results_path}/{image_name}/edges.png')
    
    circles = hough_circle_transform(edges, radius_range=(10,100))  # 参数可调
    result = draw_detected_circles(image, circles)
    result.save(f'{results_path}/{image_name}/result.png')

Detected circles: [(160, 205, 20), (118, 276, 22), (350, 129, 20), (47, 270, 29), (221, 201, 25), (45, 205, 20), (105, 203, 23), (349, 45, 30), (362, 194, 33), (283, 123, 25), (254, 274, 25), (106, 130, 20), (46, 129, 21), (162, 53, 24), (212, 129, 20), (289, 54, 20), (311, 274, 25), (225, 53, 23), (283, 200, 22), (373, 277, 23), (104, 60, 21), (162, 132, 18), (183, 271, 26)]
Detected circles: [(323, 175, 45), (296, 94, 72), (70, 168, 49), (205, 137, 61), (123, 130, 45), (341, 206, 10), (358, 179, 11), (204, 170, 10), (305, 203, 12)]
Detected circles: [(206, 46, 13), (291, 213, 14), (305, 54, 13), (125, 40, 13), (136, 163, 14), (237, 185, 13), (228, 126, 13), (190, 110, 13), (260, 144, 13), (133, 129, 13), (205, 176, 13), (248, 83, 13), (199, 140, 13), (284, 167, 14), (164, 77, 14)]


In [8]:
import cv2
from PIL import Image
import numpy as np

def detect_coins(image_path):
    # 读取图像
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # # 高斯模糊
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # Canny边缘检测
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)
    # 显示边缘检测的结果
    cv2.imshow('Edges', edges)

    # 等待按键事件，然后关闭窗口
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # 霍夫圆变换圆检测
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, minDist=20,
                               param1=50, param2=30, minRadius=10, maxRadius=100)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = (i[0], i[1])  # 圆心(x, y)
            radius = i[2]  # 半径
            cv2.circle(img, center, radius, (255, 0, 0), 2)
            cv2.circle(img, center, 2, (0, 255, 0), 3)
    
    cv2.imshow('Detected Coins', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 使用示例
image_path = './data/test_images/image.png'
detect_coins(image_path)


[[[ 45.5 258.5  27.8]
  [270.5 118.5  24.6]
  [299.5 264.5  25.7]
  [101.5 195.5  21.1]
  [153.5 197.5  20. ]
  [215.5  50.5  23. ]
  [ 44.5 196.5  18.8]
  [337.5 124.5  19.4]
  [173.5 261.5  26.8]
  [154.5  50.5  23.2]
  [115.5 265.5  21. ]
  [ 45.5 124.5  20.2]
  [213.5 193.5  24. ]
  [276.5  52.5  19.5]
  [271.5 192.5  21.6]
  [357.5 267.5  20.8]
  [206.5 123.5  20.1]
  [103.5 125.5  19.1]
  [100.5  55.5  18.8]
  [152.5 126.5  19.3]
  [242.5 264.5  23. ]
  [337.5  45.5  27.9]
  [352.5 188.5  27.8]]]
